In [1]:
!pip install fastapi nest_asyncio uvicorn pyngrok diffusers transformers torch accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [2]:
!ngrok config add-authtoken 

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:

from io import BytesIO
import nest_asyncio, threading, base64, tempfile
from typing import List
from fastapi import FastAPI
from pydantic import BaseModel
from diffusers import StableDiffusionXLPipeline
from pyngrok import ngrok
import uvicorn, torch
import torch
import os
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

# ---------- load SDXL‑Turbo (GPU) ----------------------------------------
pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=DTYPE,          # ← use the real dtype
        variant="fp16"              # keeps weights in half‑precision
).to("cuda" if torch.cuda.is_available() else "cpu")


# ---------- FastAPI app ---------------------------------------------------
class SceneReq(BaseModel):
    scene_number : int
    image_prompt : str

app = FastAPI(title="Image‑Only Worker")

@app.post("/generate_images")
def generate_images(scenes: List[SceneReq]):
    os.makedirs("generated", exist_ok=True)
    out = {}
    for s in scenes:
        try:
            img = pipe(s.image_prompt, num_inference_steps=25, height=1000, width=1600).images[0]
            # Save locally
            local_path = f"generated/scene_{s.scene_number:02}.png"
            img.save(local_path)
            # Also return base64
            buf = BytesIO()
            img.save(buf, format="PNG")
            buf.seek(0)
            out[s.scene_number] = base64.b64encode(buf.read()).decode()
        except Exception as e:
            print(f"❌ Error in scene {s.scene_number}: {e}")
    return out




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
# --- replace the “expose” block in Colab A and Colab B ---

import nest_asyncio, threading, time, socket, uvicorn
from pyngrok import ngrok
nest_asyncio.apply()

PORT = 9002

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")

# 1️⃣ Start the server
threading.Thread(target=run_uvicorn, daemon=True).start()

# 2️⃣ Wait until the port is listening
for _ in range(30):                        # ~15 s max
    with socket.socket() as s:
        if s.connect_ex(("127.0.0.1", PORT)) == 0:
            break
    time.sleep(0.5)

# 3️⃣ Create the tunnel
public_url = ngrok.connect(PORT, bind_tls=False).public_url
print("🌐  Public URL:", public_url)


INFO:     Started server process [409]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9002 (Press CTRL+C to quit)


🌐  Public URL: http://b517-34-142-252-137.ngrok-free.app


<IPython.core.display.Javascript object>

In [ ]:
%%javascript
function keepAlive() {
  setInterval(() => {
    google.colab.kernel.invokeFunction('notebook.ping', [], {});
    console.log("⏳ Keeping Colab alive...");
  }, 60000);  // every 60 seconds
}
keepAlive();